In [ ]:
def find_overlapping_for_F2(interval1, interval2):
    if interval1[0] < interval2[0]:
        if interval1[1] > interval2[0]:
            if interval1[1] < interval2[1]:
                dif = interval1[1] - interval2[0]
                return dif
            if interval1[1] > interval2[1]:
                dif = interval2[1] - interval2[0]
                return dif
    if interval1[0] > interval2[0]:
        if interval2[1] > interval1[0]:
            if interval2[1] < interval1[1]:
                dif = interval2[1] - interval1[0]
                return dif
            if interval2[1] < interval1[1]:
                dif = interval1[1] - interval1[0]
                return dif
    if interval1[0] == interval2[0]:
        if interval2[1] == interval1[1]:
            return interval1[1] - interval1[0]
        if interval2[1] > interval1[1]:
            return interval1[1] - interval1[0]
        if interval2[1] < interval1[0]:
            return interval2[1] - interval2[0]
    return 0

In [ ]:
#这个函数的box的格式是 = [x,y,x1,y1] or [c,r,c1,c2] x，y就是annotations里面的x，y x1 = x + width y1 = y + height
def IoU(ground_truth_box,submission_box):
    g_interval_x = [ground_truth_box[0],ground_truth_box[2]]
    g_interval_y = [ground_truth_box[1],ground_truth_box[3]]
    s_interval_x = [submission_box[0],submission_box[2]]
    s_interval_y = [submission_box[1],submission_box[3]]
    intersect_x = find_overlapping_for_F2(g_interval_x,s_interval_x)
    intersect_y = find_overlapping_for_F2(g_interval_y,s_interval_y)
    g_area = (ground_truth_box[2]-ground_truth_box[0])*(ground_truth_box[3]-ground_truth_box[1])
    s_area = (submission_box[2]-submission_box[0])*(submission_box[3]-submission_box[1])
    if intersect_x > 0 and intersect_y > 0:
        area_intersect = intersect_y*intersect_x
        iou = area_intersect/(g_area+s_area-area_intersect)
        return iou
    return 0

In [ ]:
#这个函数的box的格式是[(x,y),(x+width,y+height)] x,y,width,height 都是annotations里面的
#B就是 annotations里面的全部框 是ground truth
#S是你在一张图片里面检测出来的全部框
def JudgeDetection(B,S,threshold):
    TP = 0
    FP = 0
    matched_B = []
    for box_s in S:
        max_iou = 0
        max_iou_ground_truth_box = []
        for box_g in B:
            ground_truth_box = [box_g[0][0],box_g[0][1],box_g[1][0],box_g[1][1]]
            submission_box = [box_s[0][0],box_s[0][1],box_s[1][0],box_s[1][1]]
            iou = IoU(ground_truth_box,submission_box)
            if iou > 0 and iou > max_iou:
                max_iou_ground_truth_box = [ground_truth_box]
                max_iou = iou
        if max_iou >= threshold:
            TP += 1
        elif max_iou_ground_truth_box and max_iou < threshold:
            FP += 1
        if max_iou_ground_truth_box:
           matched_B.append(max_iou_ground_truth_box[0])
    FN =  len(B) - len(matched_B)
    return TP, FP, FN

In [ ]:
def F2_score(TP,FP,FN):
    F2 = (1+4)*TP/((1+4)*TP+4*FN+FP)
    return F2